In [1]:
import numpy as np # generic math functions
import matplotlib.pyplot as plt
#%matplotlib notebook
%matplotlib inline

In [2]:
from quspin.operators import commutator
from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import spin_basis_1d # Hilbert space spin basis
import numpy as np # generic math functions

### Sanity check

In [3]:
def Ham(L):
    basis = spin_basis_1d(L)
    hz=5.0
    hx=10.0
    hz_arr = [[hz,i] for i in range(L)] # OBC
    hx_arr = [[hx,i] for i in range(L)] # OBC
    # static and dynamic lists
    static = [["z",hz_arr], ["x",hx_arr] ]
    dynamic =[]
    H = hamiltonian(static,dynamic,basis=basis,dtype=np.complex_)
    return H


def del_lambda_Ham_single(L):
    basis = spin_basis_1d(L)
    hx_lamb=1.0
    hx_lamb_arr = [[hx_lamb,i] for i in range(L)] # OBC
    static_lamb = [["x",hx_lamb_arr]]
    dynamic_lamb =[]
    op_lamb=hamiltonian(static_lamb,dynamic_lamb,basis=basis,dtype=np.complex_)
    return op_lamb

In [11]:
L=1
H=Ham(L)
partial_lamb_H=del_lambda_Ham_single(L)
print commutator(H,partial_lamb_H).toarray()
#mat=((-1j*comm)**2).toarray()
#print np.trace(mat)/2**L, 4*2*(L-1)

Hermiticity check passed!
Symmetry checks passed!
Hermiticity check passed!
Symmetry checks passed!
[[  0.+0.j -10.+0.j]
 [ 10.+0.j   0.+0.j]]


In [8]:
print partial_lamb_H.toarray()
print H.toarray()

[[ 0.+0.j  1.+0.j]
 [ 1.+0.j  0.+0.j]]
[[ -5.+0.j  10.+0.j]
 [ 10.+0.j   5.+0.j]]


In [12]:
lambda_H=partial_lamb_H.toarray()
H=H.toarray()    

In [15]:
np.dot(lambda_H, H)-np.dot( H, lambda_H)

array([[  0.+0.j,  10.+0.j],
       [-10.+0.j,   0.+0.j]])

### Approximate gauge potential

In [19]:
def Ham_nonint(L):
    basis = spin_basis_1d(L)
    hz=(np.sqrt(5)+1)/4 #parameters used by Kim and Huse
    hx=(np.sqrt(5)+5)/8
    J=1.0
    hz_arr = [[hz,i] for i in range(L)] 
    hx_arr = [[hx,i] for i in range(L)] 
    J_arr =[[J,i,(i+1)] for i in range(L-1)] # OBC  [[J,i,(i+1)%L] for i in range(L)] # PBC
    
    # static and dynamic lists
    static = [["zz",J_arr],["z",hz_arr], ["x",hx_arr] ]
    dynamic =[]
    H = hamiltonian(static,dynamic,basis=basis,dtype=np.float_,check_symm=False,check_herm=False)
    return H

def del_lambda_Ham(L):
    basis = spin_basis_1d(L)  
    hx_lamb=1.0
    hx_lamb_arr = [[hx_lamb,i] for i in range(L)] 
    static_lamb = [["x",hx_lamb_arr]]
    dynamic_lamb =[]
    op_lamb=hamiltonian(static_lamb,dynamic_lamb,basis=basis,dtype=np.float_,check_symm=False,check_herm=False)
    return op_lamb

def term(comm, mu, n):
    return (-1)**n*comm/mu**(2*n+2)

def norm(A_lamb):    
    return np.linalg.norm(A_lamb, 'fro')

In [21]:
L=4
mu=1

####
####
L=2
norm_arr=np.zeros(5)

for j in range(5):
    sum_temp=0
    H=Ham_nonint(L)
    partial_lamb_H=del_lambda_Ham(L)
    for i in range(L+1):
        comm=commutator(H,partial_lamb_H).toarray()
        sum_temp+=term(comm,mu,i)
    norm_arr[j]=norm(sum_temp)
    L+=1
    print L, norm_arr[j]

3 7.27628146171
4 0.0
5 23.4851905523
6 0.0
7 59.7103429205
